## Robot@Home 2 - Captured data `v1.2`

`R@H2 notebook series`   

<a href="https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/40-Captured-data.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>



### R@H2 functions introduced in this notebook:



-   `rh.get_sensor_observations(set_name)`
-   `rh.id2name(id, arg)`
-   `rh.name2id(name, arg)`
-   `time_win2unixepoch(win_t)`
-   `time_unixepoch2win(unix_t)`



### Initial requirements



Install Robot@Home2 Toolbox using the Python package manager



In [ ]:
!pip install robotathome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 836 kB 13.4 MB/s 
     |████████████████████████████████| 58 kB 1.3 MB/s 


Check that the installation has been successful



In [ ]:
import robotathome as rh
print (rh.__version__)

1.0.5


#### Using Google Colab and Google Drive



If you haven't yet mounted R@H2 Dataset public share on your Google Drive click
[this](https://drive.google.com/drive/folders/15fQwm4G3hHQR5vI9q0CVDlwxMwaJheC8?usp=sharing).

In order to access from Google Colab it's mandatory to create a `shorcut`. You
just need to select the shared folder `Shared with me > R@H2-2.0.1`, right click
and select the corresponding menu option `Add Shortcut to Drive`. Now, a new
folder (actually a shortcut) with the same name will appear under `My Drive`.

Next, mount Google Drive in Google Colab. The browser will ask for the
authorization to access the Google Drive account. Sign in to your Google account
and Google Drive will be mounted to `/content/drive`. Note that the R@H2 files are
located in the `/content/drive/MyDrive/R@H2-2.0.1` folder.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


More info in [this notebook](https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/05-Google-colab-drive.ipynb).



### Importing R@H2



In [ ]:
import os
import pandas as pd
from robotathome import RobotAtHome
from robotathome import logger, log
from robotathome import time_win2unixepoch, time_unixepoch2win

In [ ]:
log.set_log_level('INFO')  # SUCCESS is the default

level_no, level_name = log.get_current_log_level()
print(f'Current log level name: {level_name}')

Current log level name: INFO


### Instantiating the toolbox



The toolbox is modeled around the RobotAtHome class, so we'll begin instantiating it and
creating a working object.

The RobotAtHome class has some attributes:

-   `rh_path` : string with the full path for robotathome database, usually `rh.db`.
    Its default value is `'.'` (the current directory)
-   `wspc_path` : string with the full path where results or temporary files are
    stored. It's the path to the workspace directory. Its default value also is
    `'.'`.
-   `db_filename` : string with the database filename. Its default value is `'rh.db'`
-   `rgbd_path` : string with the full path to where RGBD images are stored. Its default value is
    `'./files/rgbd'`.
-   `scene_path` : string with the full path to where Scene (3D) images are stored. Its
    default value is `'./files/scene'`.

We assume that we are running this notebook in Google Colab. Therefore our
folder structure is as follows:

    /content/drive
    └─── MyDrive
         ├─── R@H2-2.0.1
         │    └── files
         │        ├── rgbd
         │        └── scene
         └─────── rh.db

Let's instantiate our first, and usually the only one, object:



In [ ]:
my_rh_path = '/content/drive/MyDrive/R@H2-2.0.1'
my_rgbd_path = os.path.join(my_rh_path, 'files/rgbd')
my_scene_path = os.path.join(my_rh_path, 'files/scene')
my_wspc_path = '/content'

try: 
      rh = RobotAtHome(my_rh_path, my_rgbd_path, my_scene_path, my_wspc_path)
except:
      logger.error("Something was wrong")

2022-06-05 00:44:29.787 | SUCCESS  | robotathome.core.reader:__open_dataset:85 - Connection is established: rh.db


Now you are ready to dive into the R@H2 database. Following we'll show a set of
functions to extract some of the main reference data.



### Introduction



In the previous notebook, the spatial structure (homes and rooms) and the
temporal structure (sessions and sub-sessions) of the data set have been
presented. In addition, the sensors (cameras and lasers) used for data capture
have been presented.

In this notebook we are going to show how these data are organized, which we
call sensor observations.



### Sensor observations



As you know from the previous notebook the data provided by R@H2 have been
collected within **rooms** of five dwellings (**homes**) Raw data were collected in
different **sessions** and **sub-sessions**, each one containing a number of sequences
of **RGB-D** observations and **laser scans**. These sequences were gathered by
teleoperating the robot to fully inspect each individual room.

Sensor data comprises ~75 minutes of recorded data collected in different
sessions.

These data include:

-   RGB-D observations from the four RGB-D cameras, including intensity images,
    depth images, and 3D point clouds.

-   Laser scanner data: 2D observations from the laser scanner captured in the
    inspected rooms.

The R@H2 dataset contains those RGBD and Laser observations



In [ ]:
# The full dataset is returned by default
full = rh.get_sensor_observations()
print(f"# Full set: {len(full)} observations with {len(full.columns)} fields")

# Full set: 116418 observations with 14 fields


Sensor observations are ordered by `timestamp` and have the following fields



In [ ]:
print(full.columns.tolist())

['id', 'timestamp', 'home_session_id', 'home_subsession_id', 'home_id', 'room_id', 'sensor_id', 'sensor_name', 'sensor_pose_x', 'sensor_pose_y', 'sensor_pose_z', 'sensor_pose_yaw', 'sensor_pose_pitch', 'sensor_pose_roll']


The field names describe themselves

    |--------------------+--------------------|
    | field name         | description        |
    |--------------------+--------------------|
    | id                 | observation id     |
    | timestamp          | timestamp          |
    | home_session_id    | home session id    |
    | home_subsession_id | home subsession id |
    | home_id            | home id            |
    | room_id            | room id            |
    | sensor_id          | sensor id          |
    | sensor_name        | sensor name        |
    | sensor_pose_x      | sensor pose x      |
    | sensor_pose_y      | sensor pose y      |
    | sensor_pose_z      | sensor pose z      |
    | sensor_pose_yaw    | sensor pose yaw    |
    | sensor_pose_pitch  | sensor pose pitch  |
    | sensor_pose_roll   | sensor pose roll   |
    |--------------------+--------------------|

As it's a pandas data frame you can get more info with the method `info()`.



In [ ]:
print(full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116418 entries, 0 to 116417
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  116418 non-null  int64  
 1   timestamp           116418 non-null  int64  
 2   home_session_id     116418 non-null  int64  
 3   home_subsession_id  116418 non-null  int64  
 4   home_id             116418 non-null  int64  
 5   room_id             116418 non-null  int64  
 6   sensor_id           116418 non-null  int64  
 7   sensor_name         116418 non-null  object 
 8   sensor_pose_x       116418 non-null  float64
 9   sensor_pose_y       116418 non-null  float64
 10  sensor_pose_z       116418 non-null  float64
 11  sensor_pose_yaw     116418 non-null  float64
 12  sensor_pose_pitch   116418 non-null  float64
 13  sensor_pose_roll    116418 non-null  float64
dtypes: float64(6), int64(7), object(1)
memory usage: 12.4+ MB
None


It can be observed that the full set contains 
`116418` observations.

The info for each sensor observation can be organized as

    |--------------------+----------------------------------------------|
    | field group        | description                                  |
    |--------------------+----------------------------------------------|
    | id, timestamp      | main reference                               |
    | <whatever>_id      | environment reference                        |
    | sensor id and name | sensor reference                             |
    | sensor pose        | pose x,y,z,yaw, pitch and roll of the sensor |
    |--------------------+----------------------------------------------|

As an example, let's examine the first sensor observation



In [ ]:
print(full.loc[1])

id                                237014
timestamp             130488627193178472
home_session_id                        4
home_subsession_id                     0
home_id                                4
room_id                               40
sensor_id                              0
sensor_name                      HOKUYO1
sensor_pose_x                      0.205
sensor_pose_y                        0.0
sensor_pose_z                       0.31
sensor_pose_yaw                      0.0
sensor_pose_pitch                    0.0
sensor_pose_roll                     0.0
Name: 1, dtype: object


Please note that the timestamp is in Windows format but can be easily
transformed in Unix epoch format (a more human readable format)



In [ ]:
win_t = full.loc[1]['timestamp']
unix_t = time_win2unixepoch(win_t)
print(f"windows:{win_t} -> unix:{unix_t}")

windows:130488627193178472 -> unix:2014-07-03 12:05:19.317846


You can also get the names corresponding to ids. Let's associate
the home session name for the previous observation:



In [ ]:
# Get the home sessions
hs = rh.get_home_sessions()
# Get the home session id from the first observation 
so_hs_id = full.loc[1]['home_session_id']
# Access the cell containins the name.
# row: id, column: name
so_hs_name = hs.loc[so_hs_id]['name']
print(f"The home session name for the home session id {so_hs_id} is {so_hs_name}")

The home session name for the home session id 4 is sarmis-s1


We know your time is very important so a function is provided to easily do the
same thing



In [ ]:
so_hs_name = rh.id2name(so_hs_id,'hs')
print(f"The home session name for the home session id {so_hs_id} is {so_hs_name}")

The home session name for the home session id 4 is sarmis-s1


The first parameter of `id2name` function is the id, and the second one is a
selector which can adopt the following values

    |-------+---------------|
    | short | long          |
    |-------+---------------|
    | 'h'   | 'home'        |
    | 'hs'  | 'home_session |
    | 'r'   | 'room'        |
    | 'rt'  | 'room_type'   |
    | 's'   | 'sensor'      |
    | 'st'  | 'sensor_type' |
    | 'o'   | 'object'      |
    | 'ot'  | 'object_type' |
    |-------+---------------|



### Dissecting the sensor observations set



Not all observations are equally useful. Specifically, there are observations
that have been discarded for their processing. As an example, think about ank
unfocused image or a black image.

The full set in the previous section includes all captured data but other
subsets can be extracted



#### Labeled RGBD observations



Labeled RGBD observations include per-pixel object labels (category and
instance) within each RGB-D observation, i.e., both intensity and depth images,
and per-point labels within their respective point clouds.



In [ ]:
# The full dataset is returned by default
lblrgbd = rh.get_sensor_observations('lblrgbd')
print(f"# Labeled RGBD set: {len(lblrgbd)} observations with {len(lblrgbd.columns)} fields")

# Labeled RGBD set: 32937 observations with 14 fields


Note that the id of these observations is a value in the range of 100000 and
`132937`



#### Laser scanner observations



Laser scanner are 2D observations from the laser scanner captured in the
inspected rooms



In [ ]:
# The full dataset is returned by default
lsrscan = rh.get_sensor_observations('lsrscan')
print(f"# Laser scanner set: {len(lsrscan)} observations with {len(lsrscan.columns)} fields")

# Laser scanner set: 39363 observations with 14 fields


Note that the id of these observations is a value in the range of 200000 and
`239363`



#### Discarded observations



All the observation no included in previous sets, i.e., the discarded
observations can also be extracted



In [ ]:
# The full dataset is returned by default
discarded = rh.get_sensor_observations('discarded')
print(f"# Discarded set: {len(discarded)} observations with {len(discarded.columns)} fields")

# Discarded set: 44118 observations with 14 fields


This set includes all RGBD and Laser scan discarded observations. These
observations can be useful for other works.

Note that the id of these observations is a value in the range of 1 and
`44118`



#### Labeled RGBD + Laser Scan



It is common to work with a set of all processed observations, aka RGBD and
Laser scan observations



In [ ]:
# The full dataset is returned by default
rgbdlsr = rh.get_sensor_observations('rgbdlsr')
print(f"# RGBD & Laser scanner set: {len(rgbdlsr)} observations with {len(rgbdlsr.columns)} fields")

# RGBD & Laser scanner set: 72300 observations with 14 fields


Just for your information, as you know the sensor observations sets are data frames so you
can easily join them together



In [ ]:
rgbd_lsr = pd.concat([lblrgbd, lsrscan]).sort_values(by=['timestamp'])
print(f"# RGBD & Laser scanner set: {len(rgbd_lsr)} observations with {len(rgbd_lsr.columns)} fields")

# RGBD & Laser scanner set: 72300 observations with 14 fields


<a href="https://colab.research.google.com/github/goyoambrosio/RobotAtHome2/blob/master/notebooks/40-Captured-data.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

